In [1]:
import os

In [2]:
%pwd

'/home/rgukt/Desktop/NLPproj/Text_Summerization-NLP/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/home/rgukt/Desktop/NLPproj/Text_Summerization-NLP'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [11]:
from textSummerizer.constants import *
from textSummerizer.utils.common import read_yaml,create_directories 

ImportError: cannot import name 'logger' from 'textSummerizer.logging' (/home/rgukt/Desktop/NLPproj/Text_Summerization-NLP/src/textSummerizer/logging/__init__.py)

In [ ]:
Class 